In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

In [2]:
test=pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv')

In [3]:
train=pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv')

In [4]:
train.describe()

In [5]:
X_train=[]
for i in range(len(train)):
    image = tf.keras.preprocessing.image.load_img('../input/gtsrb-german-traffic-sign/'+str(train['Path'][i]),grayscale=False, color_mode='rgb', target_size=(25,25),
    interpolation='nearest')
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])
    X_train.append(input_arr)

In [6]:
X_train=np.array(X_train)

In [7]:
X_train.shape

In [8]:
X_train=X_train.reshape((39209,25, 25, 3))

In [9]:
Y_train=train['ClassId']

In [10]:
data_augmentation=models.Sequential([
    layers.experimental.preprocessing.RandomContrast(0.85,input_shape=(25,25,3)),
    layers.experimental.preprocessing.RandomZoom(0.2)
])

In [11]:
model = models.Sequential([
                      data_augmentation,
                      layers.Conv2D(filters=16,kernel_size=(3,3),activation='relu'),
                      layers.MaxPooling2D((2,2)),
                       layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu'),
                       layers.MaxPooling2D((2,2)),
                        layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
                        layers.MaxPooling2D((2,2)),
                       layers.Flatten(),
                      layers.Dense(256,activation='relu'),
                      layers.Dense(128,activation='relu'),
                      layers.Dense(64,activation='softmax')])


In [12]:
model.summary()

In [13]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
history = model.fit(X_train,Y_train,epochs=20)

In [14]:
X_test=[]
for i in range(len(test)):
    image = tf.keras.preprocessing.image.load_img('../input/gtsrb-german-traffic-sign/'+str(test['Path'][i]),grayscale=False, color_mode='rgb', target_size=(25,25),
    interpolation='nearest')
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])
    X_test.append(input_arr)

In [16]:
X_test=np.array(X_test)

In [17]:
X_test.shape

In [18]:
X_test=X_test.reshape(12630,25,25,3)

In [21]:
Y_test=test.ClassId

In [22]:
model.evaluate(X_test,Y_test)

In [23]:
prediction = model.predict(X_test)

In [24]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [25]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

In [26]:
prediction_classes=[np.argmax(element) for element in prediction]
print("Classfication report\n",classification_report(Y_test,prediction_classes))

In [28]:
def prediction(img):
    img=np.array(img)
    img=cv2.resize(img,(25,25))
    img=np.array(img).reshape(-1,25,25,3)
    return np.argmax(model.predict(img))

In [41]:
img=cv2.imread('../input/gtsrb-german-traffic-sign/Meta/7.png')

In [42]:
prediction(img)